In [1]:
from IPython.display import Image 
from pyramid.response import Response


In [2]:
from ZODB import FileStorage, DB
from ZODB.interfaces import IBlobStorage
import BTrees.OOBTree
import transaction
import persistent
from datetime import datetime
from datetime import timezone
import numpy as np
from ZODB.blob import Blob
from PIL import Image as PImage
import os

In [3]:
from CameraDetails import CameraDetails
from CaptureDetails import CaptureDetails
from Result import Result
from Settings import Settings
from Trial import Trial
from User import User

In [4]:
from random import randint
def makeCameraDetails():
    return CameraDetails(randint(1,10),randint(1,10),randint(1,10),randint(1,10),randint(1,10),randint(1,10),randint(1,10),randint(1,10),randint(1,10),randint(1,10))
def makeCaptureDetails():
    sp = randint(500,6000)
    duration = int(randint(sp,1200000)/1000) #0 to 1200 seconds (20 mins)
    interval = int(randint(sp,duration*1000)/1000)
    count = duration/interval
    return CaptureDetails(sp,duration,interval,count)
def makeResult():
    return Result(randint(0,1),randint(500,700),randint(0,10))
def makeImage():
    A = (np.random.rand(1080,1920,3)*255).astype('uint8')
    im = PImage.fromarray(A)
    return im

def makeScanPaths(trialId,userId,count):
    return [f"data/user{userId}/trial{trialId}/scan{i}.jpg" for i in range(count)]

def makeSettings():
    return Settings(randint(0,100),randint(0,100),randint(0,100),randint(0,1600),randint(0,200000))
def makeUser(userId,name):
    pin = 1234
    userType = randint(0,2)
    active = True
    settings = makeSettings()
    return User(userId,name,pin,active,userType, settings)


In [22]:
def generateUser(userId,name):
    return makeUser(userId,name,randint(1,3))

def generateTrial(trialId,userId,generateImages=False):
    #userId, dateTime, notes, scan, result, captureDetails, cameraDetails
    paths = makeScanPaths(trialId,userId,randint(1,3))
    if (generateImages):
        for path in paths:
            img = makeImage()
            if not os.path.isdir(path.split('/s')[0]):
                os.makedirs(path.split('/s')[0])
            img.save(path)
    return Trial(trialId,userId,datetime.now().astimezone(timezone.utc).timestamp(),
                 makeScanPaths(trialId,userId,randint(1,3)),str(userId) + ": Good notes",makeResult(),
                 makeCaptureDetails(),makeCameraDetails())

In [6]:
storage = FileStorage.FileStorage('LucereDummy.fs')
db = DB(storage)
connection = db.open()
root = connection.root()

In [7]:
names = ["Jonah", "Vanessa", "Ciprian", "Sadie", "Atticus"]
users = []
trials = []
for userId in range(len(names)):
    print(names[userId])
    u = generateUser(userId,names[userId])
    users.append(u)
    trials.append(generateTrial(len(trials),userId,generateImages=True))
    

Jonah
Vanessa
Ciprian
Sadie
Atticus


In [23]:
for user in users:
    print(f'User:{user.userId},{user.name}: {user.userType}')
    print(user.trialIds)
    print(user.get_settings().format_settings())

User:0,Jonah: 2


AttributeError: 'User' object has no attribute 'trialIds'

In [9]:
for trial in trials:
    print(trial.format_trial())

Trial: 0 : 0 : 2021-04-01 16:39:48.639786 
 Result: Detected: 1, Wavelength: 539, Strength: 10 
 Path: ['data/user0/trial0/scan0.jpg', 'data/user0/trial0/scan1.jpg']
Trial: 1 : 1 : 2021-04-01 16:39:49.017953 
 Result: Detected: 0, Wavelength: 589, Strength: 1 
 Path: ['data/user1/trial1/scan0.jpg', 'data/user1/trial1/scan1.jpg']
Trial: 2 : 2 : 2021-04-01 16:39:49.145952 
 Result: Detected: 1, Wavelength: 622, Strength: 1 
 Path: ['data/user2/trial2/scan0.jpg']
Trial: 3 : 3 : 2021-04-01 16:39:49.274952 
 Result: Detected: 0, Wavelength: 631, Strength: 9 
 Path: ['data/user3/trial3/scan0.jpg', 'data/user3/trial3/scan1.jpg']
Trial: 4 : 4 : 2021-04-01 16:39:49.657952 
 Result: Detected: 1, Wavelength: 568, Strength: 7 
 Path: ['data/user4/trial4/scan0.jpg', 'data/user4/trial4/scan1.jpg']


In [10]:
root.users = BTrees.OOBTree.BTree()
root.trials = BTrees.OOBTree.BTree()

In [11]:
for u in users:
    root.users[u.userId] = u
for t in trials:
    root.trials[t.trialId] = t
transaction.commit()

In [12]:
##Converts blob uid to PIL image
def serveimage(uid,width,height):
    # retrieve uid from somewhere
    resp = Response(content_type='image/jpeg')
    resp.app_iter = uid.open('r')  # open for reading
    return PImage.frombytes('RGB', (width,height), resp.body, 'raw')

In [25]:
x = makeScanPaths(0,1,3)

In [15]:
len(root.trials)

5

In [19]:

userTrials= []
for t in u.trials:
    userTrials.append(root.trials[t])

Jonah
Vanessa
Ciprian
Sadie
guest


In [21]:
for trial in root.trials:
    t = root.trials[trial]
    print(t.captureDetails.shutter_speed)

2218
3509
2255
2786
5386
